In [1]:
import conllu
import pandas as pd
from tqdm import tqdm
import numpy as np
import spacy
import sys
import json

In [2]:
sys.path.append("../src/")
from features.nlp_preprocess import extract_feat_spacy

http://www.nilc.icmc.usp.br/semanticnlp/index.php?id=index&id_sub=principal&dir_sub=includes/projects/propbankbr&dir=includes/projects/propbankbr&lang=pt-br#palmeretal2005

Índice: Número da palavra na frase.
Forma: A forma original da palavra na frase.
Lema: O lema ou a forma base da palavra.
POS (Part of Speech): Categoria gramatical da palavra (por exemplo, n = substantivo, v = verbo, adj = adjetivo, etc.).
Características Morfológicas: Informações morfológicas da palavra (por exemplo, gênero e número).
ID da Cabeça (Head ID): Índice da palavra que é a cabeça da dependência.
Relação de Dependência: Relação de dependência entre a palavra e sua cabeça (por exemplo, SUBJ = sujeito, OBJ = objeto, etc.).
Verificação de Sujeito Nulo (Null Subject Check): Indica se a palavra está envolvida em uma verificação de sujeito nulo (Y = sim, N = não).
Lema do Verbo Principal (Main Verb Lemma): O lema do verbo principal da frase.
Argumento 0 (A0): Marcador do argumento 0 (frequentemente o sujeito) do verbo.
Argumento 1 (A1): Marcador do argumento 1 (frequentemente o objeto) do verbo.
Argumento 2 (A2): Marcador do argumento 2 do verbo (se aplicável).
Argumento Modificador (AM-*): Marcadores de modificadores adicionais (por exemplo, AM-LOC para localização, AM-ADV para advérbio, etc.).

In [3]:
path = '../data/raw/propbankbr_v1.1_conll/PropBankBr_v1.1_Dep.conll.txt'

# Open the plain text file for reading; assign under 'data'
with open(path, mode="r", encoding="utf-8") as data:
    
    # Read the file contents and assign under 'annotations'
    annotations = data.read()

In [4]:
sents = annotations.split("\n\n")
sents = [sent for sent in sents if sent != '']

In [5]:
list_dict_an = []
list_dicts = []
propbank_id = 0
corpus_name = "PropBankBr_v1.1_Dep"
for sent in tqdm(sents):
    
    propbank_id += 1
    
    dict_an = {}

    tks = sent.split("\n")
    tks = [tk for tk in tks if tk != ""]
    
    list_id = []
    list_form = []
    list_lemma = []
    list_postag = []
    list_morph = []
    list_head_id = []
    list_deprel = []
    list_null_sujeito = []
    list_main_verb_lemma = []

    args_list = []
    
    
    dict_an_snt = {}
    for id_tk, tk in enumerate(tks):

        tk_an = tk.split()

        # pega as 10 caracteristicas (colunas) fixas (as restantes sao args)
        id, form, lemma, postag, morph, head_id, deprel, null_sujeito, main_verb_lemma  = tk_an[:9]
        
        id = int(id)
        form = form.replace(" ", "")
        lemma = lemma.replace(" ", "")
        postag = postag.replace(" ", "")
        morph = morph.replace(" ", "")
        head_id = head_id.replace(" ", "") 
        deprel = deprel.replace(" ", "") 
        null_sujeito = null_sujeito.replace(" ", "")
        main_verb_lemma = main_verb_lemma.replace(" ", "")
        
        
        list_id.append(id)
        list_form.append(form)
        list_lemma.append(lemma)
        list_postag.append(postag) 
        list_morph.append(morph) 
        list_head_id.append(head_id) 
        list_deprel.append(deprel) 
        list_null_sujeito.append(null_sujeito) 
        list_main_verb_lemma.append(main_verb_lemma) 

        # get args 
        args = tk_an[9:len(tk_an)]
        # format  
        args = [arg.replace(" ", "")  for arg in args]
        
        
        
        args_list.append(np.array(args))
        
        dict_an_snt.update({
            id_tk:{
                "id": id,
                "form": form,
                "lemma": lemma,
                "postag": postag, 
                "morph": morph, 
                "head_id": head_id, 
                "deprel": deprel, 
                "null_sujeito": null_sujeito, 
                "main_verb_lemma": main_verb_lemma
            }
        })

    sentence = " ".join(list_form)
    
    
    dict_an.update({"id": "propbank" + "_" + str(propbank_id)})
    dict_an.update({"snt":sentence})
    dict_an.update({"annotated_sentence_gold":dict_an_snt})
    
    tuples_root_verb = [(i, verb) for i,  verb in enumerate(list_main_verb_lemma) if verb != "-"]
    
    roots_position, root_verbs = list(map(list, zip(*tuples_root_verb)))

    
    if -1 in roots_position:
        print("Não foi achado o root")
        
        
    args_mtx = np.array(args_list)
    args_df = pd.DataFrame(args_mtx, columns = [f"{root_verbs[i]}_{roots_position[i]}" for i in range(len(root_verbs))], index=list_id)
    # replace * with nan (all the values are ARGS or V now)
    args_df.replace('-', np.nan, inplace=True)
    
    args_df.dropna(how="all", inplace = True)

    not_root_position = [id - 1 for id in args_df.index.tolist()]

    nodes_tk_ids = set(not_root_position + roots_position)


    # cria dicionarios com os nos e tambem dicionario que converte posicao para node
    nodes = {}
    node_id = 0
    pos_to_node = {}
    dict_aligments = {}
    for tk_id in nodes_tk_ids:
        
        node_id+=1
        
        nodes.update({
            node_id: list_lemma[tk_id]
        })
        
        pos_to_node.update({tk_id:node_id})
        dict_aligments.update({node_id: tk_id})

    dict_an.update({"nodes": nodes})
    dict_an.update({"nodes_to_position": pos_to_node})
    dict_an.update({"dict_aligments": dict_aligments})


    edge_id = 0
    edges = {}

    for col in args_df.columns:
        
        root_verb, root_verb_position = col.split("_")
        
        args_verb = args_df[col].dropna()

        for idx, arg in args_verb.items():
            
            tk_position = idx - 1
            
            lemma = list_lemma[tk_position]
            
            edge_id+=1
            edges.update({
                f"edge {edge_id}": {
                    "nodes_ids": (str(pos_to_node[int(root_verb_position)]), str(pos_to_node[tk_position])),
                    "nodes": (root_verb, lemma),
                    "value": "ARG0" if "A0" in arg else "ARG1" if "A1" in arg else "ARG2" if "A2" in arg else "ARG3" if "A3" in arg else "ARG4" if "A4" in arg else "ARG5" if "A5" in arg else arg
                }
            })
            
    dict_an.update({"edges": edges})
    list_dicts.append(dict_an)
    
    
nlp_model_spacy = spacy.load("pt_core_news_lg")
print('Anotando texto ...')    
for dict_an in tqdm(list_dicts):
    
    dict_spacy = extract_feat_spacy(
        text = dict_an['snt'],
        nlp_model = nlp_model_spacy)
    
            
    dict_an.update(dict_spacy)
    #dict_an.update({'tok pos': tok_pos})
    dict_an.update({"corpus_name": corpus_name})
    
    list_dict_an.append(dict_an)


with open('../data/processed/annotated_text_propbank.json', 'w') as f:
    json.dump(list_dict_an, f)

  5%|▌         | 176/3348 [00:00<00:03, 876.95it/s]/tmp/ipykernel_821327/2707196193.py:98: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  args_df.replace('-', np.nan, inplace=True)
/tmp/ipykernel_821327/2707196193.py:98: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  args_df.replace('-', np.nan, inplace=True)
  8%|▊         | 265/3348 [00:00<00:03, 882.57it/s]/tmp/ipykernel_821327/2707196193.py:98: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, expl

Anotando texto ...


100%|██████████| 3348/3348 [00:22<00:00, 149.28it/s]
